Some trivial examples to demonstrate core principles of the language, displaying other syntax along the way.

# Functions

Let's define a function:

In [1]:
function fact(n)
    if n == 1
        n
    else
        n * fact(n-1)
    end
end

fact (generic function with 1 method)

In [26]:
fact(15)

1307674368000

In [76]:
using Test  # standard library package
@test fact(15) == Base.factorial(15)

Test Passed

Julia's JIT works like a static compiler, and generates high-quality machine code:

In [2]:
@code_native fact(15)

	.text
	pushq	%rbx
	movq	%rdi, %rbx
	cmpq	$1, %rbx
	jne	L17
	movl	$1, %eax
	popq	%rbx
	retq
L17:
	leaq	-1(%rbx), %rdi
	movabsq	$fact, %rax
	callq	*%rax
	imulq	%rbx, %rax
	popq	%rbx
	retq
	nopw	(%rax,%rax)


In [32]:
fact(30)

-8764578968847253504

In [33]:
typeof(ans)

Int64

Being fast means compromises: no `BigInt`s *by default*:

In [51]:
fact(big(30))

265252859812191058636308480000000

In [52]:
fact(30.0)

2.6525285981219103e32

In [3]:
@code_native fact(30.0)

	.text
	pushq	%rax
	movabsq	$140291786693712, %rax  # imm = 0x7F983A1C5C50
	vucomisd	(%rax), %xmm0
	jne	L19
	jnp	L57
L19:
	movabsq	$140291786693720, %rax  # imm = 0x7F983A1C5C58
	vmovsd	%xmm0, (%rsp)
	vaddsd	(%rax), %xmm0, %xmm0
	movabsq	$fact, %rax
	callq	*%rax
	vmulsd	(%rsp), %xmm0, %xmm0
	popq	%rax
	retq
L57:
	vmovsd	(%rax), %xmm0           # xmm0 = mem[0],zero
	popq	%rax
	retq
	nop


## Why start with functions?

Functions are important!
* compiled by the JIT
* known local state

In [29]:
using BenchmarkTools  # external package, more on this later

In [46]:
a = 1
b = 2
@btime sin(a)+b;

  40.129 ns (2 allocations: 32 bytes)


In [47]:
myfunction(a, b) = sin(a)+b
@btime myfunction(1,2);

  7.700 ns (0 allocations: 0 bytes)


# Macro's

* functions: work with values
* macros: work with expressions

Very powerful! Only describe shortly to explain their use.

In [38]:
a = [1, 2, 3]

3-element Array{Int64,1}:
 1
 2
 3

In [44]:
acc = 0
for i in a
    @show acc += i
end
@test acc == sum(a)

┌ Warning: Deprecated syntax `implicit assignment to global variable `acc``.
│ Use `global acc` instead.
└ @ nothing none:0


acc += i = 1
acc += i = 3
acc += i = 6


Test Passed

In [45]:
?@show

```
@show
```

Show an expression and result, returning the result.


# Types

In [10]:
typeof(42)

Int64

In [12]:
Int === Int64

true

In [11]:
T = Int
while T !== supertype(T)
    T = @show supertype(T)
end

supertype(T) = Signed
supertype(T) = Integer
supertype(T) = Real
supertype(T) = Number
supertype(T) = Any


In [13]:
struct SpecialNumber <: Number
    x::Int
end

In [14]:
x = SpecialNumber(42)

SpecialNumber(42)

In [48]:
[SpecialNumber(i) for i in 1:10]

10-element Array{SpecialNumber,1}:
  SpecialNumber(1)
  SpecialNumber(2)
  SpecialNumber(3)
  SpecialNumber(4)
  SpecialNumber(5)
  SpecialNumber(6)
  SpecialNumber(7)
  SpecialNumber(8)
  SpecialNumber(9)
 SpecialNumber(10)

In [74]:
special_mat = SpecialNumber.(rand(1:100, 10, 10))

10×10 Array{SpecialNumber,2}:
  SpecialNumber(92)  SpecialNumber(76)  …  SpecialNumber(37)
 SpecialNumber(100)  SpecialNumber(89)     SpecialNumber(80)
  SpecialNumber(67)  SpecialNumber(71)      SpecialNumber(7)
  SpecialNumber(34)  SpecialNumber(73)      SpecialNumber(4)
  SpecialNumber(15)  SpecialNumber(19)      SpecialNumber(5)
  SpecialNumber(81)  SpecialNumber(99)  …  SpecialNumber(26)
  SpecialNumber(76)  SpecialNumber(76)     SpecialNumber(93)
  SpecialNumber(12)  SpecialNumber(48)     SpecialNumber(97)
  SpecialNumber(78)  SpecialNumber(77)     SpecialNumber(79)
  SpecialNumber(28)  SpecialNumber(98)     SpecialNumber(39)

# Multiple dispatch

Single-dispatch languages: `SpecialNumber::plus(SpecialNumber other)`

In [77]:
function Base.:+(a::SpecialNumber, b::SpecialNumber)
    SpecialNumber(a.x + b.x)
end

In [29]:
x + x

SpecialNumber(84)

In [34]:
Base.:+(a::SpecialNumber, b::Number) = SpecialNumber(a.x + b)
Base.:+(a::Number, b::SpecialNumber) = SpecialNumber(a + b.x)

In [35]:
x + 1

SpecialNumber(43)

In [36]:
1 + x

SpecialNumber(43)

In [39]:
sum(SpecialNumber[x, x, x])

SpecialNumber(126)

In [69]:
Base.:*(a::SpecialNumber, b::SpecialNumber) = SpecialNumber(a.x * b.x)
special_mat * special_mat

5×5 Array{SpecialNumber,2}:
 SpecialNumber(11961)  SpecialNumber(21506)  …   SpecialNumber(9653)
 SpecialNumber(12228)  SpecialNumber(29248)     SpecialNumber(15268)
 SpecialNumber(12679)  SpecialNumber(31501)     SpecialNumber(15950)
 SpecialNumber(14362)  SpecialNumber(29526)     SpecialNumber(13284)
 SpecialNumber(10942)  SpecialNumber(21297)     SpecialNumber(10317)